In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import os
import time

In [2]:
# Setup the Open-Meteo API client with cache
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define the API URL
url = "https://archive-api.open-meteo.com/v1/archive"

# Load latitude and longitude data from a CSV file
locations_df = pd.read_csv(r'D:\_Bootcamp\_Project4\Final_Cleaned_Data\Airport_Hub_List.csv')  
output_folder = r"D:\_Bootcamp\_Project4\Final_Cleaned_Data\Weather_Data\Weather_2022"  # Output folder location


# Prepare to collect results
api_requests_made = 0  # Counter to track the number of API requests made

try:
    # Loop through each location in the CSV
    for index, row in locations_df.iterrows():
        latitude = row['LAT_DECIMAL']
        longitude = row['LONG_DECIMAL']
        arpt_id = row['ARPT_ID']  # Use ARPT_ID column for file naming
        
        print(f"Processing location {index + 1}/{len(locations_df)}: ARPT_ID={arpt_id}, Latitude={latitude}, Longitude={longitude}")
        
        # Define parameters for the API request
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": "2022-01-01",
            "end_date": "2022-12-31",
            "daily": [
                "weather_code", "temperature_2m_max", "temperature_2m_min", "precipitation_sum",
                "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max",
                "wind_gusts_10m_max", "wind_direction_10m_dominant"
            ]
        }
        
        # Make the API request
        responses = openmeteo.weather_api(url, params=params)
        api_requests_made += 1  # Increment API request counter
        
        # Process the response data
        response = responses[0]
        daily = response.Daily()
        
        # Extract daily weather data
        daily_data = {
            "date": pd.date_range(
                start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=daily.Interval()),
                inclusive="left"
            ),
            "latitude": latitude,
            "longitude": longitude,
            "weather_code": daily.Variables(0).ValuesAsNumpy(),
            "temperature_2m_max": daily.Variables(1).ValuesAsNumpy(),
            "temperature_2m_min": daily.Variables(2).ValuesAsNumpy(),
            "precipitation_sum": daily.Variables(3).ValuesAsNumpy(),
            "snowfall_sum": daily.Variables(5).ValuesAsNumpy(),
            "precipitation_hours": daily.Variables(6).ValuesAsNumpy(),
            "wind_speed_10m_max": daily.Variables(7).ValuesAsNumpy(),
            "wind_gusts_10m_max": daily.Variables(8).ValuesAsNumpy(),
            "wind_direction_10m_dominant": daily.Variables(9).ValuesAsNumpy()
        }
        
        # Convert to DataFrame
        daily_dataframe = pd.DataFrame(data=daily_data)
        
        # Save to CSV file named after ARPT_ID
        output_file = os.path.join(output_folder, f"{arpt_id}_daily_weather_data.csv")
        daily_dataframe.to_csv(output_file, index=False)
        print(f"Data saved to '{output_file}'.")

        # Display API request count every 2 requests
        if api_requests_made % 2 == 0:
            print(f"API requests made: {api_requests_made}. Pausing for 10 seconds...")
            time.sleep(10)  # Pause for 10 seconds after every 10 requests

except KeyboardInterrupt:
    print("\nProcess interrupted by user.")
finally:
    print(f"Weather API requests completed.")

Processing location 1/125: ARPT_ID=ATL, Latitude=33.63669961, Longitude=-84.427864
Data saved to 'D:\_Bootcamp\_Project4\Final_Cleaned_Data\Weather_Data\Weather_2022\ATL_daily_weather_data.csv'.
Processing location 2/125: ARPT_ID=LAX, Latitude=33.94249638, Longitude=-118.40804861
Data saved to 'D:\_Bootcamp\_Project4\Final_Cleaned_Data\Weather_Data\Weather_2022\LAX_daily_weather_data.csv'.
API requests made: 2. Pausing for 10 seconds...
Processing location 3/125: ARPT_ID=DFW, Latitude=32.89723305, Longitude=-97.03769472
Data saved to 'D:\_Bootcamp\_Project4\Final_Cleaned_Data\Weather_Data\Weather_2022\DFW_daily_weather_data.csv'.
Processing location 4/125: ARPT_ID=SFO, Latitude=37.61880555, Longitude=-122.37541666
Data saved to 'D:\_Bootcamp\_Project4\Final_Cleaned_Data\Weather_Data\Weather_2022\SFO_daily_weather_data.csv'.
API requests made: 4. Pausing for 10 seconds...
Processing location 5/125: ARPT_ID=DEN, Latitude=39.86166666, Longitude=-104.67316666
Data saved to 'D:\_Bootcamp\_P

OpenMeteoRequestsError: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}